<a href="https://colab.research.google.com/github/samiha-mahin/Data-Analysis/blob/main/FE_Sklearn_Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [17]:
!pip install -U scikit-learn

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [19]:
df = pd.read_csv('/content/titanic.csv')

In [20]:
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


**Drop Unnecessary Columns**


 These columns are likely:

* PassengerId: Just an identifier, not useful for prediction.

* Name: Might contain useful info, but needs complex text processing.

* Ticket: No consistent structure.

* Cabin: Too many missing values (NaNs).

* So, to simplify, you drop them.







In [21]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'], inplace=True)

In [22]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),
                                                 df['Survived'],
                                                 test_size=0.2,
                                                 random_state=30)

**✅ Example: X_train.head()**

| Pclass | Sex    | Age  | SibSp | Parch | Fare    | Embarked |
| ------ | ------ | ---- | ----- | ----- | ------- | -------- |
| 3      | male   | 22.0 | 1     | 0     | 7.2500  | S        |
| 1      | female | 38.0 | 1     | 0     | 71.2833 | C        |
| 3      | female | 26.0 | 0     | 0     | 7.9250  | S        |
| 1      | female | 35.0 | 1     | 0     | 53.1000 | S        |
| 3      | male   | 35.0 | 0     | 0     | 8.0500  | S        |


**✅ Example: y_train.head()**

| Survived |
| -------- |
| 0        |
| 1        |
| 1        |
| 1        |
| 0        |

Each value corresponds to the survival status of the matching row in X_train.

**✅ Example: X_test.head()**

| Pclass | Sex    | Age  | SibSp | Parch | Fare   | Embarked |
| ------ | ------ | ---- | ----- | ----- | ------ | -------- |
| 3      | male   | 28.0 | 0     | 0     | 8.4583 | Q        |
| 2      | female | 19.0 | 0     | 0     | 13.000 | S        |
| 1      | male   | 45.0 | 1     | 1     | 83.475 | S        |
| 3      | male   | NaN  | 0     | 0     | 7.2250 | C        |
| 2      | female | 30.0 | 0     | 0     | 12.350 | Q        |

**✅ Example: y_test.head()**

| Survived |
| -------- |
| 0        |
| 1        |
| 1        |
| 0        |
| 1        |



In [23]:
X_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
199,2,female,24.0,0,0,13.0000,S
703,3,male,25.0,0,0,7.7417,Q


In [24]:
y_train.sample(5)

,Survived
94,0
679,1
861,0
532,0
558,1


In [25]:
#imputation transformer
trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]), #Name of this transformation: impute_age , Column index 2 → Age column
    ('impute_embarked',SimpleImputer(strategy ='most_frequent'),[6]) #this fills missing embarkation values with the most common (mode) value.
],remainder='passthrough')

In [30]:
# one hot encoding
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(handle_unknown='ignore'),[1,6]) #Column 1 → Sex (male, female),Column 6 → Embarked (C, Q, S)
],remainder='passthrough')

**"Apply MinMaxScaler to the first 10 columns."**

After previous transformations (like one-hot encoding), your dataset might have many more columns than before — especially because Sex and Embarked were turned into multiple binary columns.

In [29]:
# Scaling
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])